[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/notebooks/blob/main/camenduru's_tensorrt_diffusers.ipynb)

In [ ]:
%cd /content
!mkdir /content/external
%cd /content/external
!git clone https://github.com/pybind/pybind11
!wget https://www.python.org/ftp/python/3.10.12/Python-3.10.12.tgz
!tar -xvf Python-3.10.12.tgz
!mkdir -p /content/external/python3.10/include
!cp -r /content/external/Python-3.10.12/Include/* /content/external/python3.10/include
!wget http://http.us.debian.org/debian/pool/main/p/python3.10/libpython3.10-dev_3.10.13-1_amd64.deb
!ar x libpython3.10-dev*.deb
!mkdir /content/external/debian
!tar -xf data.tar.xz -C /content/external/debian
!cp /content/external/debian/usr/include/x86_64-linux-gnu/python3.10/pyconfig.h /content/external/python3.10/include
%cd /content/TensorRT/python
!TENSORRT_MODULE=tensorrt PYTHON_MAJOR_VERSION=3 PYTHON_MINOR_VERSION=10 TARGET_ARCHITECTURE=x86_64 ./build.sh

In [ ]:
%cd /content
!git clone -b release/9.1 --recursive https://github.com/NVIDIA/TensorRT
%cd /content/TensorRT

%cd /content
!wget https://developer.nvidia.com/downloads/compute/machine-learning/tensorrt/secure/9.1.0/tars/tensorrt-9.1.0.4.linux.x86_64-gnu.cuda-11.8.tar.gz
!mkdir trt
%cd /content/trt
!tar -xvzf /content/tensorrt-9.1.0.4.linux.x86_64-gnu.cuda-11.8.tar.gz

%cd /content/TensorRT
!mkdir -p /content/TensorRT/build
%cd /content/TensorRT/build
!cmake .. -DTRT_LIB_DIR=/content/trt/TensorRT-9.1.0.4/lib -DTRT_OUT_DIR=/content/TensorRT/build/out -DCUDNN_ROOT_DIR=/usr/local/cuda -DGPU_ARCHS="75"
!make -j$(nproc)
# !/content/TensorRT/build/out/trtexec -h

In [ ]:
# !pip install -q diffusers transformers accelerate colored cuda-python ftfy nvtx controlnet_aux
# !pip install -q --extra-index-url https://pypi.ngc.nvidia.com "onnx" "onnxruntime" "onnx-graphsurgeon" "polygraphy"
!pip install -q diffusers==0.19.3 transformers==4.31.0 accelerate colored cuda-python ftfy nvtx controlnet_aux==0.0.6 onnx==1.14.0 onnxruntime==1.15.1 -U
!pip install -q --extra-index-url https://pypi.ngc.nvidia.com "onnx-graphsurgeon>=0.3.27" "polygraphy" -U
!pip install -q https://huggingface.co/tensorrt/build/resolve/main/tensorrt-9.1.0.post11.dev4-cp310-none-linux_x86_64.whl -U

In [ ]:
%env LD_LIBRARY_PATH=/content/trt/TensorRT-9.1.0.4/lib:/usr/lib64-nvidia
# !echo $LD_LIBRARY_PATH
%cd /content/TensorRT/demo/Diffusion
# !LD_PRELOAD=/content/trt/TensorRT-9.1.0.4/lib/libnvinfer.so.9 python demo_txt2img_xl.py -h
# !LD_PRELOAD=/content/trt/TensorRT-9.1.0.4/lib/libnvinfer.so.9 python demo_txt2img_xl.py "a beautiful photograph of Mt. Fuji during cherry blossom"
!LD_PRELOAD=/content/trt/TensorRT-9.1.0.4/lib/libnvinfer.so.9 python demo_txt2img.py "a beautiful photograph of Mt. Fuji during cherry blossom" # sd1.5

In [ ]:
# https://github.com/NVIDIA/TensorRT/blob/release/8.6/demo/experimental/HuggingFace-Diffusers/TensorRT-diffusers-txt2img.ipynb
# https://github.com/huggingface/diffusers/tree/main/examples/community#tensorrt-text2image-stable-diffusion-pipeline
# https://github.com/huggingface/diffusers/tree/main/examples/community#tensorrt-image2image-stable-diffusion-pipeline
# https://github.com/huggingface/diffusers/tree/main/examples/community#tensorrt-inpainting-stable-diffusion-pipeline

# https://github.com/huggingface/diffusers/blob/main/examples/community/stable_diffusion_tensorrt_txt2img.py
# https://github.com/huggingface/diffusers/blob/main/examples/community/stable_diffusion_tensorrt_img2img.py
# https://github.com/huggingface/diffusers/blob/main/examples/community/stable_diffusion_tensorrt_inpaint.py
# https://github.com/huggingface/diffusers/blob/main/examples/community/run_tensorrt_controlnet.py

!pip install -q tensorrt diffusers transformers accelerate
!pip install -q --extra-index-url https://pypi.ngc.nvidia.com "onnx-graphsurgeon" "onnxruntime" "polygraphy"

In [ ]:
import torch
from diffusers.pipelines.stable_diffusion import StableDiffusionPipeline
from diffusers import DDIMScheduler

scheduler = DDIMScheduler.from_pretrained("stabilityai/stable-diffusion-2-1", subfolder="scheduler")
pipe_trt = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1", custom_pipeline="stable_diffusion_tensorrt_txt2img", revision='fp16',
    torch_dtype=torch.float16, scheduler=scheduler, image_height=512, image_width=512)

pipe_trt.set_cached_folder("stabilityai/stable-diffusion-2-1", revision='fp16')

pipe_trt = pipe_trt.to("cuda")

In [ ]:
prompt = "a beautiful photograph of Mt. Fuji during cherry blossom"

# warm up runs to stabilize performance benchmarking
num_warm_up_steps=5
for _ in range(num_warm_up_steps):
    _ = pipe_trt(prompt)

image = pipe_trt(prompt).images[0]
display(image)